In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip
/kaggle/input/word2vec-nlp-tutorial/sampleSubmission.csv
/kaggle/input/word2vec-nlp-tutorial/unlabeledTrainData.tsv.zip
/kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip
/kaggle/input/imdb-master-csv-zip/imdb_master.csv


In [2]:
# Suppressing warnings
import warnings
warnings.filterwarnings('ignore')

# Importing necessary libraries
import numpy as np
import pandas as pd
import re
import os
import html
import matplotlib.pyplot as plt
import seaborn as sns
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec, Phrases
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Bidirectional
from tensorflow.keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from xgboost import XGBClassifier

2024-02-09 21:50:25.311782: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-09 21:50:25.311836: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-09 21:50:25.313367: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Initializing classifiers
random_forest = RandomForestClassifier()
gradient_boosting = GradientBoostingClassifier()
decision_tree = DecisionTreeClassifier()
logistic_regression = LogisticRegression()
k_neighbors = KNeighborsClassifier()
gaussian_naive_bayes = GaussianNB()
bernoulli_naive_bayes = BernoulliNB()
xgboost_classifier = XGBClassifier()
multinomial_naive_bayes = MultinomialNB()

In [4]:
# Define file paths
train_data_path = "../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip"
test_data_path = "../input/word2vec-nlp-tutorial/testData.tsv.zip"
additional_data_csv_path = "../input/imdb-master-csv-zip/imdb_master.csv"

# Reading datasets
train = pd.read_csv(train_data_path, sep='\t')
test = pd.read_csv(test_data_path, sep='\t')

# Reading additional CSV file
att = pd.read_csv(additional_data_csv_path, encoding='latin')

In [5]:
# Selecting relevant columns and renaming them
att = att[["review", "label"]]
att.columns = ["review", "sentiment"]

# Removing rows with "unsup" sentiment and converting sentiment to binary
att = att[att.sentiment != "unsup"]
att["sentiment"] = att["sentiment"].map({"neg": 0, "pos": 1})

# Displaying the first few rows
att.head()

,review,sentiment
0,Once again Mr. Costner has dragged out a movie...,0
1,This is an example of why the majority of acti...,0
2,"First of all I hate those moronic rappers, who...",0
3,Not even the Beatles could write songs everyon...,0
4,Brass pictures (movies is not a fitting word f...,0


In [6]:
# Getting the shapes of the datasets
train_shape = train.shape
test_shape = test.shape
att_shape = att.shape

# Displaying the shapes
train_shape, test_shape, att_shape

((25000, 3), (25000, 2), (50000, 2))

In [7]:
# Concatenating the datasets
df = pd.concat([train, att, test], ignore_index=True)

# Displaying information about the combined DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   id         50000 non-null   object 
 1   sentiment  75000 non-null   float64
 2   review     100000 non-null  object 
dtypes: float64(1), object(2)
memory usage: 2.3+ MB


In [8]:
# Displaying the first few rows of the combined DataFrame
df.head()

,id,sentiment,review
0,5814_8,1.0,With all this stuff going down at the moment w...
1,2381_9,1.0,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0.0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0.0,It must be assumed that those who praised this...
4,9495_8,1.0,Superbly trashy and wondrously unpretentious 8...


In [9]:
df.review=html.unescape(df.review)
# removing html symbols

df.review=df.review.str.replace('http\S+|www.\S+', '', case=False).str.replace(r"\&\#[0-9]+\;","", regex=True) \
.str.replace(r"[^\w\s]","").str.replace("\d+","").str.replace(r"[\s]+"," ",regex=True) \
.str.replace("\n"," ").replace("\r","").str.replace("_"," ").str.lower()

In [10]:
def remove_emoji(text):
    # Define the emoji pattern using Unicode ranges
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    # Remove emojis from the text using the pattern
    clean_text = emoji_pattern.sub(r'', text)
    return clean_text

In [11]:
from unicodedata import normalize

def remove_accent(text):
    # Normalize the text to decompose and separate accents
    normalized_text = normalize("NFKD", text)
    # Encode the normalized text to ASCII, ignoring any characters that cannot be represented in ASCII
    ascii_text = normalized_text.encode("ascii", "ignore")
    # Decode the ASCII text back to UTF-8, ignoring any decoding errors
    clean_text = ascii_text.decode("utf-8", "ignore")
    return clean_text

# Apply the remove_accent function to the 'review' column of the DataFrame
df["review"] = df["review"].apply(remove_accent)

In [12]:
# Display the first few rows of the DataFrame
print("First few rows of the DataFrame:")
print(df.head())

# Display the value counts of the 'sentiment' column
print("\nValue counts of the 'sentiment' column:")
print(df['sentiment'].value_counts())

First few rows of the DataFrame:
       id  sentiment                                             review
0  5814_8        1.0  with all this stuff going down at the moment w...
1  2381_9        1.0  \the classic war of the worlds\" by timothy hi...
2  7759_3        0.0  the film starts with a manager (nicholas bell)...
3  3630_4        0.0  it must be assumed that those who praised this...
4  9495_8        1.0  superbly trashy and wondrously unpretentious 8...

Value counts of the 'sentiment' column:
sentiment
1.0    37500
0.0    37500
Name: count, dtype: int64


In [13]:
corpora = df["review"].values
tokenized = [word_tokenize(corpus) for corpus in corpora]

print(tokenized[2222])

['go', 'immediately', 'and', 'rent', 'this', 'movie', '.', 'it', 'will', 'be', 'be', 'on', 'a', 'bottom', 'shelf', 'in', 'your', 'local', 'video', 'store', 'and', 'will', 'be', 'covered', 'in', 'dust', '.', 'no', 'one', 'will', 'have', 'touched', 'it', 'in', 'years', '.', 'it', 'may', 'even', 'be', 'a', '$', '.50', 'special', '!', 'it', "'s", 'worth', 'ten', 'bucks', ',', 'i', 'swear', '!', 'buy', 'it', '!', 'there', 'are', "n't", 'very', 'many', 'films', 'than', 'can', 'compare', 'with', 'this', '-', 'the', 'celluloid', 'version', 'of', 'that', 'goo', 'that', 'forms', 'at', 'the', 'bottom', 'of', 'a', 'trash', 'can', 'after', 'a', 'few', 'years', '.', 'yes', ',', 'i', 'gave', 'it', 'a', "'1", ',', "'", 'but', 'it', 'really', 'deserves', 'much', 'lower', '.', '1-10', 'scales', 'were', 'not', 'designed', 'with', 'stuff', 'like', 'this', 'in', 'mind', '.']


In [14]:
tokenized = [list(filter(lambda x: len(x) > 1, document)) \
             for document in tokenized]

print(tokenized[2222])

['go', 'immediately', 'and', 'rent', 'this', 'movie', 'it', 'will', 'be', 'be', 'on', 'bottom', 'shelf', 'in', 'your', 'local', 'video', 'store', 'and', 'will', 'be', 'covered', 'in', 'dust', 'no', 'one', 'will', 'have', 'touched', 'it', 'in', 'years', 'it', 'may', 'even', 'be', '.50', 'special', 'it', "'s", 'worth', 'ten', 'bucks', 'swear', 'buy', 'it', 'there', 'are', "n't", 'very', 'many', 'films', 'than', 'can', 'compare', 'with', 'this', 'the', 'celluloid', 'version', 'of', 'that', 'goo', 'that', 'forms', 'at', 'the', 'bottom', 'of', 'trash', 'can', 'after', 'few', 'years', 'yes', 'gave', 'it', "'1", 'but', 'it', 'really', 'deserves', 'much', 'lower', '1-10', 'scales', 'were', 'not', 'designed', 'with', 'stuff', 'like', 'this', 'in', 'mind']


In [15]:
%%time
from gensim.models import word2vec
import numpy as np

# Set numpy print options to suppress scientific notation
np.set_printoptions(suppress=True)

# Define parameters for Word2Vec model
feature_size = 256
context_size = 5
min_word = 1
epochs = 50
random_seed = 42

# Train Word2Vec model
word_vec_model = word2vec.Word2Vec(tokenized, 
                                   vector_size=feature_size, 
                                   window=context_size, 
                                   min_count=min_word, 
                                   epochs=epochs, 
                                   seed=random_seed)

# Print the time taken for training
print("Word2Vec model training time:")

Word2Vec model training time:
CPU times: user 1h 55s, sys: 13.9 s, total: 1h 1min 8s
Wall time: 21min 43s


In [18]:
from keras.preprocessing.text import Tokenizer

# Initialize the Tokenizer
tokenizer = Tokenizer()

# Fit the tokenizer on the tokenized text
tokenizer.fit_on_texts(tokenized)

# Convert the tokenized text to sequences of integers
sequences = tokenizer.texts_to_sequences(tokenized)

# Padding sequences to ensure uniform length
padded_sequences = pad_sequences(sequences, padding='post')

# Converting the padded sequences into a NumPy array
tokenized_array = np.array(padded_sequences)

# Checking the shape of the tokenized array
print("Shape of tokenized array:", tokenized_array.shape)

Shape of tokenized array: (100000, 2419)


In [ ]:
# Creating a document vector by taking the mean of word vectors in each document
model_array = np.array([word_vec_model.wv[doc].mean(axis=0) for doc in tokenized_array])

# Creating DataFrame with document vectors and sentiment labels
model_df = pd.DataFrame(model_array)
model_df["sentiment"] = df["sentiment"]

# Displaying the first few rows of the DataFrame
print("First few rows of the DataFrame:")
print(model_df.head())

In [ ]:
# Split the DataFrame into training and testing sets
df_train = model_df[:75000]
df_test = model_df[75000:]

# Confirm the sizes of the training and testing sets
print("Size of training set:", len(df_train))
print("Size of testing set:", len(df_test))

In [ ]:
# Extract the 'sentiment' column as the target variable for the training set
y = df_train.pop("sentiment")

# Extract the remaining columns as the features for the training set
text = df_train

# Remove the 'sentiment' column from the testing set
test = df_test.drop(columns=["sentiment"])

# Display the shapes of the training features, target variable, and testing features
print("Shapes of training features, target variable, and testing features:")
print("Training features shape:", text.shape)
print("Target variable shape:", y.shape)
print("Testing features shape:", test.shape)

In [ ]:
# Fit the model using the training features and target variable
r.fit(text, y)

# Make predictions on the testing set
pred = r.predict(test)

# Convert the predictions to integers
pred = pred.astype(int)

# Display the predictions
print("Predicted sentiments:")
print(pred)

In [ ]:
# Create a new DataFrame for submission
submission_df = pd.DataFrame()

# Extract 'id' column from the testing set
submission_df["id"] = df.iloc[75000:]["id"]

# Add predicted sentiments to the submission DataFrame
submission_df["sentiment"] = pred

# Display the submission DataFrame
print("Submission DataFrame:")
print(submission_df)

# Save the submission DataFrame to a CSV file
submission_df.to_csv("submission.csv", index=False)